# TCN Classification with MPQA Dataset
<hr>

We will build a text classification model using TCN model on the MPQA Dataset. Since there is no standard train/test split for this dataset, we will use 10-Fold Cross Validation (CV). 


## Load the library

In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import nltk
import random
# from nltk.tokenize import TweetTokenizer
from sklearn.model_selection import KFold

%config IPCompleter.greedy=True
%config IPCompleter.use_jedi=False
# nltk.download('twitter_samples')

In [2]:
tf.config.list_physical_devices('GPU') 

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

## Load the Dataset

In [3]:
corpus = pd.read_pickle('../../../0_data/MPQA/MPQA.pkl')
corpus.label = corpus.label.astype(int)
print(corpus.shape)
corpus

(10606, 3)


sentence  label  split
0              complaining      0  train
1       failing to support      0  train
2        desperately needs      0  train
3      many years of decay      0  train
4             no quick fix      0  train
...                    ...    ...    ...
10601                urged      1  train
10602       strictly abide      1  train
10603                 hope      1  train
10604       strictly abide      1  train
10605                           1  train

[10606 rows x 3 columns]

In [4]:
corpus.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10606 entries, 0 to 10605
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   sentence  10606 non-null  object
 1   label     10606 non-null  int32 
 2   split     10606 non-null  object
dtypes: int32(1), object(2)
memory usage: 207.3+ KB


In [5]:
corpus.groupby( by='label').count()

sentence  split
label                 
0          7294   7294
1          3312   3312

In [6]:
# Separate the sentences and the labels
sentences, labels = list(corpus.sentence), list(corpus.label)

In [7]:
sentences[0]

'complaining'

<!--## Split Dataset-->

# Data Preprocessing
<hr>

Preparing data for word embedding, especially for pre-trained word embedding like Word2Vec or GloVe, __don't use standard preprocessing steps like stemming or stopword removal__. Compared to our approach on cleaning the text when doing word count based feature extraction (e.g. TFIDF) such as removing stopwords, stemming etc, now we will keep these words as we do not want to lose such information that might help the model learn better.

__Tomas Mikolov__, one of the developers of Word2Vec, in _word2vec-toolkit: google groups thread., 2015_, suggests only very minimal text cleaning is required when learning a word embedding model. Sometimes, it's good to disconnect
In short, what we will do is:
- Puntuations removal
- Lower the letter case
- Tokenization

The process above will be handled by __Tokenizer__ class in TensorFlow

- <b>One way to choose the maximum sequence length is to just pick the length of the longest sentence in the training set.</b>

In [8]:
# Define a function to compute the max length of sequence
def max_length(sequences):
    '''
    input:
        sequences: a 2D list of integer sequences
    output:
        max_length: the max length of the sequences
    '''
    max_length = 0
    for i, seq in enumerate(sequences):
        length = len(seq)
        if max_length < length:
            max_length = length
    return max_length

In [9]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

trunc_type='post'
padding_type='post'
oov_tok = "<UNK>"

print("Example of sentence: ", sentences[4])

# Cleaning and Tokenization
tokenizer = Tokenizer(oov_token=oov_tok)
tokenizer.fit_on_texts(sentences)

# Turn the text into sequence
training_sequences = tokenizer.texts_to_sequences(sentences)
max_len = max_length(training_sequences)

print('Into a sequence of int:', training_sequences[4])

# Pad the sequence to have the same size
training_padded = pad_sequences(training_sequences, maxlen=max_len, padding=padding_type, truncating=trunc_type)
print('Into a padded sequence:', training_padded[4])

Example of sentence:  no quick fix
Into a sequence of int: [25, 945, 1476]
Into a padded sequence: [  25  945 1476    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0]


In [10]:
word_index = tokenizer.word_index
# See the first 10 words in the vocabulary
for i, word in enumerate(word_index):
    print(word, word_index.get(word))
    if i==9:
        break
vocab_size = len(word_index)+1
print(vocab_size)

<UNK> 1
the 2
of 3
to 4
a 5
and 6
not 7
is 8
in 9
be 10
6236


# Model 1: Embedding Random
<hr>

## TCN Model

Now, we will build Temporal Convolutional Network (CNN) models to classify encoded documents as either positive or negative.

The model takes inspiration from https://github.com/philipperemy/keras-tcn and https://www.kaggle.com/christofhenkel/temporal-convolutional-network

__Arguments__
`TCN(nb_filters=64, kernel_size=2, nb_stacks=1, dilations=[1, 2, 4, 8, 16, 32], padding='causal', use_skip_connections=False, dropout_rate=0.0, return_sequences=True, activation='relu', kernel_initializer='he_normal', use_batch_norm=False, **kwargs)`

- `nb_filters`: Integer. The number of filters to use in the convolutional layers. Would be similar to units for LSTM.
- `kernel_size`: Integer. The size of the kernel to use in each convolutional layer.
- `dilations`: List. A dilation list. Example is: [1, 2, 4, 8, 16, 32, 64].
- `nb_stacks`: Integer. The number of stacks of residual blocks to use.
- `padding`: String. The padding to use in the convolutions. 'causal' for a causal network (as in the original implementation) and - `'same' for a non-causal network.
- `use_skip_connections`: Boolean. If we want to add skip connections from input to each residual block.
- `return_sequences`: Boolean. Whether to return the last output in the output sequence, or the full sequence.
- `dropout_rate`: Float between 0 and 1. Fraction of the input units to drop.
- `activation`: The activation used in the residual blocks o = activation(x + F(x)).
- `kernel_initializer`: Initializer for the kernel weights matrix (Conv1D).
- `use_batch_norm`: Whether to use batch normalization in the residual layers or not.
- `kwargs`: Any other arguments for configuring parent class Layer. For example "name=str", Name of the model. Use unique names when using multiple TCN.

Now, we will define our TCN model as follows:
- One TCN layer with 100 filters, kernel size 1-6, and relu and tanh activation function;
- Dropout size = 0.5;
- Optimizer: Adam (The best learning algorithm so far)
- Loss function: binary cross-entropy (suited for binary classification problem)

**Note**: 
- The whole purpose of dropout layers is to tackle the problem of over-fitting and to introduce generalization to the model. Hence it is advisable to keep dropout parameter near 0.5 in hidden layers. 
- https://missinglink.ai/guides/keras/keras-conv1d-working-1d-convolutional-neural-networks-keras/

In [11]:
from tcn import TCN, tcn_full_summary
from tensorflow.keras.layers import Input, Embedding, Dense, Dropout, SpatialDropout1D
from tensorflow.keras.layers import concatenate, GlobalAveragePooling1D, GlobalMaxPooling1D
from tensorflow.keras.models import Model

def define_model(kernel_size = 3, activation='relu', input_dim = None, output_dim=300, max_length = None ):
    
    inp = Input( shape=(max_length,))
    x = Embedding(input_dim=input_dim, output_dim=output_dim, input_length=max_length)(inp)
    x = SpatialDropout1D(0.1)(x)
    
    x = TCN(128,dilations = [1, 2, 4], return_sequences=True, activation = activation, name = 'tcn1')(x)
    x = TCN(64,dilations = [1, 2, 4], return_sequences=True, activation = activation, name = 'tcn2')(x)
    
    avg_pool = GlobalAveragePooling1D()(x)
    max_pool = GlobalMaxPooling1D()(x)
    
    conc = concatenate([avg_pool, max_pool])
    conc = Dense(16, activation="relu")(conc)
    conc = Dropout(0.1)(conc)
    outp = Dense(1, activation="sigmoid")(conc)    

    model = Model(inputs=inp, outputs=outp)
    model.compile( loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
    
    return model

In [12]:
model_0 = define_model( input_dim=1000, max_length=100)
model_0.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 100)]        0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 100, 300)     300000      input_1[0][0]                    
__________________________________________________________________________________________________
spatial_dropout1d (SpatialDropo (None, 100, 300)     0           embedding[0][0]                  
__________________________________________________________________________________________________
tcn1 (TCN)                      (None, 100, 128)     279936      spatial_dropout1d[0][0]          
______________________________________________________________________________________________

In [13]:
# tcn_full_summary(model_0)

In [14]:
# class myCallback(tf.keras.callbacks.Callback):
#     # Overide the method on_epoch_end() for our benefit
#     def on_epoch_end(self, epoch, logs={}):
#         if (logs.get('accuracy') > 0.93):
#             print("\nReached 93% accuracy so cancelling training!")
#             self.model.stop_training=True


callbacks = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', min_delta=0, 
                                             patience=10, verbose=2, 
                                             mode='auto', restore_best_weights=True)

## Train and Test the Model

In [15]:
# Parameter Initialization
trunc_type='post'
padding_type='post'
oov_tok = "<UNK>"
activations = ['relu', 'tanh']
kernel_sizes = [1, 2, 3, 4, 5, 6]

columns = ['Activation', 'Filters', 'acc1', 'acc2', 'acc3', 'acc4', 'acc5', 'acc6', 'acc7', 'acc8', 'acc9', 'acc10', 'AVG']
record = pd.DataFrame(columns = columns)

# prepare cross validation with 10 splits and shuffle = True
kfold = KFold(10, True)

# Separate the sentences and the labels
sentences, labels = list(corpus.sentence), list(corpus.label)

exp = 0

for activation in activations:
    for kernel_size in kernel_sizes:
        # kfold.split() will return set indices for each split
        exp+=1
        print('-------------------------------------------')
        print('Training {}: {} activation, {} kernel size.'.format(exp, activation, kernel_size))
        print('-------------------------------------------')
        acc_list = []
        for train, test in kfold.split(sentences):
            
            train_x, test_x = [], []
            train_y, test_y = [], []
            
            for i in train:
                train_x.append(sentences[i])
                train_y.append(labels[i])

            for i in test:
                test_x.append(sentences[i])
                test_y.append(labels[i])

            # Turn the labels into a numpy array
            train_y = np.array(train_y)
            test_y = np.array(test_y)

            # encode data using
            # Cleaning and Tokenization
            tokenizer = Tokenizer(oov_token=oov_tok)
            tokenizer.fit_on_texts(train_x)

            # Turn the text into sequence
            training_sequences = tokenizer.texts_to_sequences(train_x)
            test_sequences = tokenizer.texts_to_sequences(test_x)

            max_len = max_length(training_sequences)

            # Pad the sequence to have the same size
            Xtrain = pad_sequences(training_sequences, maxlen=max_len, padding=padding_type, truncating=trunc_type)
            Xtest = pad_sequences(test_sequences, maxlen=max_len, padding=padding_type, truncating=trunc_type)

            word_index = tokenizer.word_index
            vocab_size = len(word_index)+1

            # Define the input shape
            model = define_model(kernel_size, activation, input_dim=vocab_size, max_length=max_len)

            # Train the model
            model.fit(Xtrain, train_y, batch_size=50, epochs=100, verbose=1, 
                      callbacks=[callbacks], validation_data=(Xtest, test_y))

            # evaluate the model
            loss, acc = model.evaluate(Xtest, test_y, verbose=0)
            print('Test Accuracy: {}'.format(acc*100))

            acc_list.append(acc*100)
            
        mean_acc = np.array(acc_list).mean()
        parameters = [activation, kernel_size]
        entries = parameters + acc_list + [mean_acc]

        temp = pd.DataFrame([entries], columns=columns)
        record = record.append(temp, ignore_index=True)
        print()
        print(record)
        print()

-------------------------------------------
Training 1: relu activation, 1 kernel size.
-------------------------------------------


C:\Users\Diardano Raihan\Anaconda3\envs\tf-gpu\lib\site-packages\sklearn\utils\validation.py:72: FutureWarning: Pass shuffle=True as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  "will result in an error", FutureWarning)


Epoch 1/100
191/191 [==============================] - 40s 139ms/step - loss: 0.5839 - accuracy: 0.7078 - val_loss: 0.4001 - val_accuracy: 0.8285
Epoch 2/100
191/191 [==============================] - 18s 92ms/step - loss: 0.2604 - accuracy: 0.9054 - val_loss: 0.4396 - val_accuracy: 0.8426
Epoch 3/100
191/191 [==============================] - 18s 93ms/step - loss: 0.1526 - accuracy: 0.9467 - val_loss: 0.4358 - val_accuracy: 0.8586
Epoch 4/100
191/191 [==============================] - 17s 91ms/step - loss: 0.1043 - accuracy: 0.9598 - val_loss: 0.5712 - val_accuracy: 0.8586
Epoch 5/100
191/191 [==============================] - 17s 89ms/step - loss: 0.0707 - accuracy: 0.9721 - val_loss: 0.8218 - val_accuracy: 0.8511
Epoch 6/100
191/191 [==============================] - 17s 91ms/step - loss: 0.0639 - accuracy: 0.9763 - val_loss: 0.6333 - val_accuracy: 0.8021
Epoch 7/100
191/191 [==============================] - 17s 89ms/step - loss: 0.0526 - accuracy: 0.9781 - val_loss: 0.7715 - val_a

Epoch 8/100
191/191 [==============================] - 20s 105ms/step - loss: 0.0530 - accuracy: 0.9793 - val_loss: 0.6926 - val_accuracy: 0.8313
Epoch 9/100
191/191 [==============================] - 19s 98ms/step - loss: 0.0417 - accuracy: 0.9829 - val_loss: 0.8042 - val_accuracy: 0.8002
Epoch 10/100
191/191 [==============================] - 18s 92ms/step - loss: 0.0365 - accuracy: 0.9832 - val_loss: 0.9271 - val_accuracy: 0.8539
Epoch 11/100
191/191 [==============================] - 17s 91ms/step - loss: 0.0407 - accuracy: 0.9832 - val_loss: 0.8420 - val_accuracy: 0.8464
Epoch 12/100
191/191 [==============================] - 18s 92ms/step - loss: 0.0354 - accuracy: 0.9840 - val_loss: 0.7796 - val_accuracy: 0.8181
Restoring model weights from the end of the best epoch.
Epoch 00012: early stopping
Test Accuracy: 86.33365035057068
Epoch 1/100
191/191 [==============================] - 30s 115ms/step - loss: 0.5776 - accuracy: 0.7130 - val_loss: 0.4331 - val_accuracy: 0.8332
Epoch 2/

191/191 [==============================] - 18s 93ms/step - loss: 0.1475 - accuracy: 0.9501 - val_loss: 0.4906 - val_accuracy: 0.8530
Epoch 4/100
191/191 [==============================] - 17s 90ms/step - loss: 0.0965 - accuracy: 0.9624 - val_loss: 0.5691 - val_accuracy: 0.8530
Epoch 5/100
191/191 [==============================] - 18s 94ms/step - loss: 0.0684 - accuracy: 0.9727 - val_loss: 0.5909 - val_accuracy: 0.8473
Epoch 6/100
191/191 [==============================] - 17s 90ms/step - loss: 0.0599 - accuracy: 0.9775 - val_loss: 0.6817 - val_accuracy: 0.8520
Epoch 7/100
191/191 [==============================] - 17s 90ms/step - loss: 0.0507 - accuracy: 0.9810 - val_loss: 0.9112 - val_accuracy: 0.8266
Epoch 8/100
191/191 [==============================] - 18s 93ms/step - loss: 0.0490 - accuracy: 0.9780 - val_loss: 0.8387 - val_accuracy: 0.8379
Epoch 9/100
191/191 [==============================] - 17s 89ms/step - loss: 0.0436 - accuracy: 0.9830 - val_loss: 0.9215 - val_accuracy: 0.82

...

Epoch 1/100
191/191 [==============================] - 32s 130ms/step - loss: 0.5557 - accuracy: 0.7369 - val_loss: 0.3628 - val_accuracy: 0.8549
Epoch 2/100
191/191 [==============================] - 23s 122ms/step - loss: 0.2462 - accuracy: 0.9184 - val_loss: 0.3561 - val_accuracy: 0.8633
Epoch 3/100
191/191 [==============================] - 23s 121ms/step - loss: 0.1713 - accuracy: 0.9477 - val_loss: 0.3765 - val_accuracy: 0.8662
Epoch 4/100
191/191 [==============================] - 23s 120ms/step - loss: 0.1266 - accuracy: 0.9587 - val_loss: 0.4656 - val_accuracy: 0.8124
Epoch 5/100
191/191 [==============================] - 22s 118ms/step - loss: 0.1144 - accuracy: 0.9644 - val_loss: 0.4617 - val_accuracy: 0.8662
Epoch 6/100
191/191 [==============================] - 23s 120ms/step - loss: 0.0937 - accuracy: 0.9659 - val_loss: 0.4854 - val_accuracy: 0.8530
Epoch 7/100
191/191 [==============================] - 23s 119ms/step - loss: 0.0885 - accuracy: 0.9673 - val_loss: 0.5004 -

191/191 [==============================] - 22s 114ms/step - loss: 0.1722 - accuracy: 0.9436 - val_loss: 0.3787 - val_accuracy: 0.8481
Epoch 4/100
191/191 [==============================] - 22s 114ms/step - loss: 0.1348 - accuracy: 0.9593 - val_loss: 0.3999 - val_accuracy: 0.8425
Epoch 5/100
191/191 [==============================] - 22s 113ms/step - loss: 0.1084 - accuracy: 0.9643 - val_loss: 0.5485 - val_accuracy: 0.8245
Epoch 6/100
191/191 [==============================] - 21s 112ms/step - loss: 0.1061 - accuracy: 0.9653 - val_loss: 0.4374 - val_accuracy: 0.8519
Epoch 7/100
191/191 [==============================] - 21s 112ms/step - loss: 0.0966 - accuracy: 0.9687 - val_loss: 0.4670 - val_accuracy: 0.8396
Epoch 8/100
191/191 [==============================] - 21s 111ms/step - loss: 0.0841 - accuracy: 0.9726 - val_loss: 0.6060 - val_accuracy: 0.8349
Epoch 9/100
191/191 [==============================] - 22s 113ms/step - loss: 0.0858 - accuracy: 0.9705 - val_loss: 0.5580 - val_accurac

Epoch 11/100
191/191 [==============================] - 18s 92ms/step - loss: 0.0626 - accuracy: 0.9736 - val_loss: 0.5514 - val_accuracy: 0.8464
Epoch 12/100
191/191 [==============================] - 18s 94ms/step - loss: 0.0579 - accuracy: 0.9770 - val_loss: 0.5768 - val_accuracy: 0.8633
Restoring model weights from the end of the best epoch.
Epoch 00012: early stopping
Test Accuracy: 86.6163969039917
Epoch 1/100
191/191 [==============================] - 24s 97ms/step - loss: 0.5716 - accuracy: 0.7185 - val_loss: 0.3796 - val_accuracy: 0.8435
Epoch 2/100
191/191 [==============================] - 18s 92ms/step - loss: 0.2499 - accuracy: 0.9107 - val_loss: 0.3382 - val_accuracy: 0.8624
Epoch 3/100
191/191 [==============================] - 18s 94ms/step - loss: 0.1743 - accuracy: 0.9400 - val_loss: 0.3962 - val_accuracy: 0.8596
Epoch 4/100
191/191 [==============================] - 18s 95ms/step - loss: 0.1299 - accuracy: 0.9522 - val_loss: 0.4141 - val_accuracy: 0.8680
Epoch 5/100


191/191 [==============================] - 17s 88ms/step - loss: 0.0533 - accuracy: 0.9778 - val_loss: 0.6548 - val_accuracy: 0.8426
Restoring model weights from the end of the best epoch.
Epoch 00012: early stopping
Test Accuracy: 85.10838747024536
Epoch 1/100
191/191 [==============================] - 22s 93ms/step - loss: 0.5625 - accuracy: 0.7203 - val_loss: 0.3980 - val_accuracy: 0.8360
Epoch 2/100
191/191 [==============================] - 17s 88ms/step - loss: 0.2522 - accuracy: 0.9091 - val_loss: 0.3849 - val_accuracy: 0.8398
Epoch 3/100
191/191 [==============================] - 17s 90ms/step - loss: 0.1652 - accuracy: 0.9456 - val_loss: 0.4655 - val_accuracy: 0.8294
Epoch 4/100
191/191 [==============================] - 17s 89ms/step - loss: 0.1287 - accuracy: 0.9596 - val_loss: 0.4634 - val_accuracy: 0.8303
Epoch 5/100
191/191 [==============================] - 17s 89ms/step - loss: 0.1069 - accuracy: 0.9636 - val_loss: 0.4707 - val_accuracy: 0.8285
Epoch 6/100
191/191 [====

Epoch 6/100
191/191 [==============================] - 17s 87ms/step - loss: 0.0932 - accuracy: 0.9726 - val_loss: 0.4645 - val_accuracy: 0.8445
Epoch 7/100
191/191 [==============================] - 17s 88ms/step - loss: 0.0878 - accuracy: 0.9696 - val_loss: 0.4254 - val_accuracy: 0.8718
Epoch 8/100
191/191 [==============================] - 17s 88ms/step - loss: 0.0882 - accuracy: 0.9676 - val_loss: 0.4886 - val_accuracy: 0.8671
Epoch 9/100
191/191 [==============================] - 17s 88ms/step - loss: 0.0687 - accuracy: 0.9749 - val_loss: 0.5225 - val_accuracy: 0.8417
Epoch 10/100
191/191 [==============================] - 17s 88ms/step - loss: 0.0696 - accuracy: 0.9721 - val_loss: 0.5583 - val_accuracy: 0.8492
Epoch 11/100
191/191 [==============================] - 17s 88ms/step - loss: 0.0639 - accuracy: 0.9727 - val_loss: 0.6289 - val_accuracy: 0.8473
Epoch 12/100
191/191 [==============================] - 17s 88ms/step - loss: 0.0557 - accuracy: 0.9775 - val_loss: 0.6032 - val

## Summary

In [16]:
record.sort_values(by='AVG', ascending=False)

Activation Filters       acc1       acc2       acc3       acc4       acc5  \
5        relu       6  86.616397  87.558907  85.862392  86.804903  84.825635   
1        relu       2  85.485393  85.296887  86.050898  85.673892  87.370408   
4        relu       5  87.087655  86.239398  85.673892  87.181902  87.181902   
7        tanh       2  85.391140  85.391140  86.522150  86.993402  86.616397   
3        relu       4  86.522150  85.673892  86.993402  85.108387  86.239398   
2        relu       3  85.579640  87.370408  85.108387  85.202640  86.239398   
8        tanh       3  86.804903  87.464654  84.919888  85.862392  86.050898   
0        relu       1  85.862392  86.899149  84.637135  86.333650  83.788878   
6        tanh       1  84.448636  86.333650  86.993402  86.899149  84.260130   
10       tanh       5  87.181902  86.333650  83.034873  85.673892  86.616397   
9        tanh       4  86.616397  86.710650  86.145145  85.391140  85.296887   
11       tanh       6  87.558907  85.108387  84.542882  86.050898  88.030159   

         acc6       acc7       acc8       acc9      acc10        AVG  
5   86.899149  85.000002  85.754716  87.830186  85.943395  86.309568  
1   86.239398  86.698115  87.358493  85.000002  87.547171  86.272066  
4   87.087655  85.754716  85.660380  85.754716  84.528303  86.215052  
7   84.354383  85.377359  86.509436  86.509436  87.830186  86.149503  
3   87.370408  85.943395  85.094339  86.320752  85.754716  86.102084  
2   85.579640  87.075472  86.037737  86.509436  86.132073  86.083483  
8   86.993402  84.622639  85.377359  83.962262  86.886793  85.894519  
0   85.485393  87.641507  87.169814  83.584905  87.264150  85.866697  
6   83.223373  84.811318  87.452829  86.320752  86.132073  85.687531  
10  86.804903  84.716982  85.660380  86.037737  84.811318  85.687203  
9   86.145145  83.679247  85.660380  87.452829  83.113205  85.621102  
11  87.653160  82.264149  83.301884  85.943395  84.245282  85.469911

In [17]:
record[['Activation', 'AVG']].groupby(by='Activation').max().sort_values(by='AVG', ascending=False)

AVG
Activation           
relu        86.309568
tanh        86.149503

In [42]:
report = record.sort_values(by='AVG', ascending=False)
report = report.to_excel('TCN_MPQA.xlsx', sheet_name='random')

# Model 2: Word2Vec Static

__Using and updating pre-trained embeddings__
* In this part, we will create an Embedding layer in Tensorflow Keras using a pre-trained word embedding called Word2Vec 300-d tht has been trained 100 bilion words from Google News.
* In this part,  we will leave the embeddings fixed instead of updating them (dynamic).

1. __Load `Word2Vec` Pre-trained Word Embedding__

In [20]:
from gensim.models import KeyedVectors
word2vec = KeyedVectors.load_word2vec_format('../GoogleNews-vectors-negative300.bin', binary=True)

In [21]:
# Access the dense vector value for the word 'handsome'
# word2vec.word_vec('handsome') # 0.11376953
word2vec.word_vec('cool') # 1.64062500e-01

array([ 1.64062500e-01,  1.87500000e-01, -4.10156250e-02,  1.25000000e-01,
       -3.22265625e-02,  8.69140625e-02,  1.19140625e-01, -1.26953125e-01,
        1.77001953e-02,  8.83789062e-02,  2.12402344e-02, -2.00195312e-01,
        4.83398438e-02, -1.01074219e-01, -1.89453125e-01,  2.30712891e-02,
        1.17675781e-01,  7.51953125e-02, -8.39843750e-02, -1.33666992e-02,
        1.53320312e-01,  4.08203125e-01,  3.80859375e-02,  3.36914062e-02,
       -4.02832031e-02, -6.88476562e-02,  9.03320312e-02,  2.12890625e-01,
        1.72119141e-02, -6.44531250e-02, -1.29882812e-01,  1.40625000e-01,
        2.38281250e-01,  1.37695312e-01, -1.76757812e-01, -2.71484375e-01,
       -1.36718750e-01, -1.69921875e-01, -9.15527344e-03,  3.47656250e-01,
        2.22656250e-01, -3.06640625e-01,  1.98242188e-01,  1.33789062e-01,
       -4.34570312e-02, -5.12695312e-02, -3.46679688e-02, -8.49609375e-02,
        1.01562500e-01,  1.42578125e-01, -7.95898438e-02,  1.78710938e-01,
        2.30468750e-01,  

2. __Check number of training words present in Word2Vec__

In [22]:
def training_words_in_word2vector(word_to_vec_map, word_to_index):
    '''
    input:
        word_to_vec_map: a word2vec GoogleNews-vectors-negative300.bin model loaded using gensim.models
        word_to_index: word to index mapping from training set
    '''
    
    vocab_size = len(word_to_index) + 1
    count = 0
    # Set each row "idx" of the embedding matrix to be 
    # the word vector representation of the idx'th word of the vocabulary
    for word, idx in word_to_index.items():
        if word in word_to_vec_map:
            count+=1
            
    return print('Found {} words present from {} training vocabulary in the set of pre-trained word vector'.format(count, vocab_size))

In [23]:
# Separate the sentences and the labels
sentences, labels = list(corpus.sentence), list(corpus.label)

# Cleaning and Tokenization
tokenizer = Tokenizer(oov_token=oov_tok)
tokenizer.fit_on_texts(sentences)

word_index = tokenizer.word_index
training_words_in_word2vector(word2vec, word_index)

Found 6083 words present from 6236 training vocabulary in the set of pre-trained word vector


2. __Define a `pretrained_embedding_layer` function__

In [24]:
emb_mean = word2vec.vectors.mean()
emb_std = word2vec.vectors.std()

In [25]:
from tensorflow.keras.layers import Embedding

def pretrained_embedding_matrix(word_to_vec_map, word_to_index, emb_mean, emb_std):
    '''
    input:
        word_to_vec_map: a word2vec GoogleNews-vectors-negative300.bin model loaded using gensim.models
        word_to_index: word to index mapping from training set
    '''
    np.random.seed(2021)
    
    # adding 1 to fit Keras embedding (requirement)
    vocab_size = len(word_to_index) + 1
    # define dimensionality of your pre-trained word vectors (= 300)
    emb_dim = word_to_vec_map.word_vec('handsome').shape[0]
    
    # initialize the matrix with generic normal distribution values
    embed_matrix = np.random.normal(emb_mean, emb_std, (vocab_size, emb_dim))
    
    # Set each row "idx" of the embedding matrix to be 
    # the word vector representation of the idx'th word of the vocabulary
    for word, idx in word_to_index.items():
        if word in word_to_vec_map:
            embed_matrix[idx] = word_to_vec_map.get_vector(word)
            
    return embed_matrix

In [26]:
# Test the function
w_2_i = {'<UNK>': 1, 'handsome': 2, 'cool': 3, 'shit': 4 }
em_matrix = pretrained_embedding_matrix(word2vec, w_2_i, emb_mean, emb_std)
em_matrix

array([[ 0.19468211,  0.08648376, -0.05924511, ..., -0.16683994,
        -0.09975549, -0.08595189],
       [-0.13509196, -0.07441947,  0.15388953, ..., -0.05400787,
        -0.13156594, -0.05996158],
       [ 0.11376953,  0.1796875 , -0.265625  , ..., -0.21875   ,
        -0.03930664,  0.20996094],
       [ 0.1640625 ,  0.1875    , -0.04101562, ...,  0.10888672,
        -0.01019287,  0.02075195],
       [ 0.10888672, -0.16699219,  0.08984375, ..., -0.19628906,
        -0.23144531,  0.04614258]])

## TCN Model

In [27]:
def define_model_2(kernel_size = 3, activation='relu', input_dim = None, 
                   output_dim=300, max_length = None, emb_matrix = None):
    
    inp = Input( shape=(max_length,))
    x = Embedding(input_dim=input_dim, 
                  output_dim=output_dim, 
                  input_length=max_length,
                  # Assign the embedding weight with word2vec embedding marix
                  weights = [emb_matrix],
                  # Set the weight to be not trainable (static)
                  trainable = False)(inp)
    
    x = SpatialDropout1D(0.1)(x)
    
    x = TCN(128,dilations = [1, 2, 4], return_sequences=True, activation = activation, name = 'tcn1')(x)
    x = TCN(64,dilations = [1, 2, 4], return_sequences=True, activation = activation, name = 'tcn2')(x)
    
    avg_pool = GlobalAveragePooling1D()(x)
    max_pool = GlobalMaxPooling1D()(x)
    
    conc = concatenate([avg_pool, max_pool])
    conc = Dense(16, activation="relu")(conc)
    conc = Dropout(0.1)(conc)
    outp = Dense(1, activation="sigmoid")(conc)    

    model = Model(inputs=inp, outputs=outp)
    model.compile( loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
    
    return model

In [28]:
model_0 = define_model_2( input_dim=1000, max_length=100, emb_matrix=np.random.rand(1000, 300))
model_0.summary()

Model: "model_121"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_122 (InputLayer)          [(None, 100)]        0                                            
__________________________________________________________________________________________________
embedding_121 (Embedding)       (None, 100, 300)     300000      input_122[0][0]                  
__________________________________________________________________________________________________
spatial_dropout1d_121 (SpatialD (None, 100, 300)     0           embedding_121[0][0]              
__________________________________________________________________________________________________
tcn1 (TCN)                      (None, 100, 128)     279936      spatial_dropout1d_121[0][0]      
__________________________________________________________________________________________

## Train and Test the Model

In [29]:
# class myCallback(tf.keras.callbacks.Callback):
#     # Overide the method on_epoch_end() for our benefit
#     def on_epoch_end(self, epoch, logs={}):
#         if (logs.get('accuracy') >= 0.9):
#             print("\nReached 90% accuracy so cancelling training!")
#             self.model.stop_training=True

callbacks = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', min_delta=0, 
                                             patience=10, verbose=2, 
                                             mode='auto', restore_best_weights=True)

In [30]:
# Parameter Initialization
trunc_type='post'
padding_type='post'
oov_tok = "<UNK>"
activations = ['relu']
print('Loading embedding statistics . . .')
emb_mean = emb_mean
emb_std = emb_std
print('Done!')
kernel_sizes = [1, 2, 3, 4, 5, 6]

columns = ['Activation', 'Filters', 'acc1', 'acc2', 'acc3', 'acc4', 'acc5', 'acc6', 'acc7', 'acc8', 'acc9', 'acc10', 'AVG']
record2 = pd.DataFrame(columns = columns)

# prepare cross validation with 10 splits and shuffle = True
kfold = KFold(10, True)

# Separate the sentences and the labels
sentences, labels = list(corpus.sentence), list(corpus.label)
exp = 0

for activation in activations:
    for kernel_size in kernel_sizes:
        exp+=1
        print('-------------------------------------------')
        print('Training {}: {} activation, {} kernel size.'.format(exp, activation, kernel_size))
        print('-------------------------------------------')
        
        # kfold.split() will return set indices for each split
        acc_list = []
        for train, test in kfold.split(sentences):
            
            train_x, test_x = [], []
            train_y, test_y = [], []
            
            for i in train:
                train_x.append(sentences[i])
                train_y.append(labels[i])

            for i in test:
                test_x.append(sentences[i])
                test_y.append(labels[i])

            # Turn the labels into a numpy array
            train_y = np.array(train_y)
            test_y = np.array(test_y)

            # encode data using
            # Cleaning and Tokenization
            tokenizer = Tokenizer(oov_token=oov_tok)
            tokenizer.fit_on_texts(train_x)

            # Turn the text into sequence
            training_sequences = tokenizer.texts_to_sequences(train_x)
            test_sequences = tokenizer.texts_to_sequences(test_x)

            max_len = max_length(training_sequences)

            # Pad the sequence to have the same size
            Xtrain = pad_sequences(training_sequences, maxlen=max_len, padding=padding_type, truncating=trunc_type)
            Xtest = pad_sequences(test_sequences, maxlen=max_len, padding=padding_type, truncating=trunc_type)

            word_index = tokenizer.word_index
            vocab_size = len(word_index)+1
            
            
            emb_matrix = pretrained_embedding_matrix(word2vec, word_index, emb_mean, emb_std)
            
            # Define the input shape
            model = define_model_2(kernel_size, activation, input_dim=vocab_size, 
                                 max_length=max_len, emb_matrix=emb_matrix)

            # Train the model
            model.fit(Xtrain, train_y, batch_size=50, epochs=100, verbose=1, 
                      callbacks=[callbacks], validation_data=(Xtest, test_y))

            # evaluate the model
            loss, acc = model.evaluate(Xtest, test_y, verbose=0)
            print('Test Accuracy: {}'.format(acc*100))

            acc_list.append(acc*100)
            
        mean_acc = np.array(acc_list).mean()
        parameters = [activation, kernel_size]
        entries = parameters + acc_list + [mean_acc]

        temp = pd.DataFrame([entries], columns=columns)
        record2 = record2.append(temp, ignore_index=True)
        print()
        print(record2)
        print()

Loading embedding statistics . . .
Done!
-------------------------------------------
Training 1: relu activation, 1 kernel size.
-------------------------------------------


C:\Users\Diardano Raihan\Anaconda3\envs\tf-gpu\lib\site-packages\sklearn\utils\validation.py:72: FutureWarning: Pass shuffle=True as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  "will result in an error", FutureWarning)


Epoch 1/100
191/191 [==============================] - 20s 69ms/step - loss: 0.4874 - accuracy: 0.7653 - val_loss: 0.3069 - val_accuracy: 0.8860
Epoch 2/100
191/191 [==============================] - 12s 65ms/step - loss: 0.2811 - accuracy: 0.8927 - val_loss: 0.2881 - val_accuracy: 0.8916
Epoch 3/100
191/191 [==============================] - 12s 65ms/step - loss: 0.2410 - accuracy: 0.9078 - val_loss: 0.2952 - val_accuracy: 0.8860
Epoch 4/100
191/191 [==============================] - 12s 63ms/step - loss: 0.2192 - accuracy: 0.9187 - val_loss: 0.2940 - val_accuracy: 0.8812
Epoch 5/100
191/191 [==============================] - 12s 62ms/step - loss: 0.1904 - accuracy: 0.9293 - val_loss: 0.2982 - val_accuracy: 0.8756
Epoch 6/100
191/191 [==============================] - 12s 62ms/step - loss: 0.1646 - accuracy: 0.9425 - val_loss: 0.3205 - val_accuracy: 0.8718
Epoch 7/100
191/191 [==============================] - 12s 62ms/step - loss: 0.1469 - accuracy: 0.9472 - val_loss: 0.3528 - val_ac

Epoch 8/100
191/191 [==============================] - 12s 64ms/step - loss: 0.1317 - accuracy: 0.9536 - val_loss: 0.3339 - val_accuracy: 0.8897
Epoch 9/100
191/191 [==============================] - 12s 63ms/step - loss: 0.1272 - accuracy: 0.9505 - val_loss: 0.3954 - val_accuracy: 0.8841
Epoch 10/100
191/191 [==============================] - 12s 63ms/step - loss: 0.1078 - accuracy: 0.9618 - val_loss: 0.3653 - val_accuracy: 0.8869
Epoch 11/100
191/191 [==============================] - 12s 63ms/step - loss: 0.0949 - accuracy: 0.9658 - val_loss: 0.4782 - val_accuracy: 0.8803
Epoch 12/100
191/191 [==============================] - 12s 63ms/step - loss: 0.1004 - accuracy: 0.9611 - val_loss: 0.4180 - val_accuracy: 0.8812
Epoch 13/100
191/191 [==============================] - 12s 63ms/step - loss: 0.0889 - accuracy: 0.9685 - val_loss: 0.4064 - val_accuracy: 0.8822
Epoch 14/100
191/191 [==============================] - 12s 63ms/step - loss: 0.0754 - accuracy: 0.9735 - val_loss: 0.5376 - v

Epoch 9/100
191/191 [==============================] - 12s 61ms/step - loss: 0.1053 - accuracy: 0.9588 - val_loss: 0.3749 - val_accuracy: 0.8764
Epoch 10/100
191/191 [==============================] - 12s 61ms/step - loss: 0.0981 - accuracy: 0.9634 - val_loss: 0.3903 - val_accuracy: 0.8840
Epoch 11/100
191/191 [==============================] - 12s 61ms/step - loss: 0.0868 - accuracy: 0.9676 - val_loss: 0.4730 - val_accuracy: 0.8717
Epoch 12/100
191/191 [==============================] - 12s 61ms/step - loss: 0.0817 - accuracy: 0.9693 - val_loss: 0.3728 - val_accuracy: 0.8868
Epoch 13/100
191/191 [==============================] - 12s 61ms/step - loss: 0.0695 - accuracy: 0.9740 - val_loss: 0.5793 - val_accuracy: 0.8811
Epoch 14/100
191/191 [==============================] - 12s 61ms/step - loss: 0.0804 - accuracy: 0.9704 - val_loss: 0.4883 - val_accuracy: 0.8783
Epoch 15/100
191/191 [==============================] - 12s 62ms/step - loss: 0.0776 - accuracy: 0.9706 - val_loss: 0.5443 - 

...

Epoch 5/100
191/191 [==============================] - 12s 63ms/step - loss: 0.1818 - accuracy: 0.9337 - val_loss: 0.3513 - val_accuracy: 0.8662
Epoch 6/100
191/191 [==============================] - 12s 63ms/step - loss: 0.1686 - accuracy: 0.9412 - val_loss: 0.3785 - val_accuracy: 0.8501
Epoch 7/100
191/191 [==============================] - 12s 62ms/step - loss: 0.1499 - accuracy: 0.9456 - val_loss: 0.3715 - val_accuracy: 0.8624
Epoch 8/100
191/191 [==============================] - 12s 63ms/step - loss: 0.1255 - accuracy: 0.9556 - val_loss: 0.4232 - val_accuracy: 0.8483
Epoch 9/100
191/191 [==============================] - 12s 62ms/step - loss: 0.1153 - accuracy: 0.9570 - val_loss: 0.3948 - val_accuracy: 0.8652
Epoch 10/100
191/191 [==============================] - 12s 63ms/step - loss: 0.1046 - accuracy: 0.9607 - val_loss: 0.4825 - val_accuracy: 0.8605
Epoch 11/100
191/191 [==============================] - 12s 62ms/step - loss: 0.0814 - accuracy: 0.9709 - val_loss: 0.4234 - val_

Epoch 11/100
191/191 [==============================] - 12s 63ms/step - loss: 0.0966 - accuracy: 0.9637 - val_loss: 0.4919 - val_accuracy: 0.8585
Epoch 12/100
191/191 [==============================] - 12s 63ms/step - loss: 0.0888 - accuracy: 0.9683 - val_loss: 0.4154 - val_accuracy: 0.8623
Epoch 13/100
191/191 [==============================] - 12s 62ms/step - loss: 0.0839 - accuracy: 0.9690 - val_loss: 0.5272 - val_accuracy: 0.8660
Epoch 14/100
191/191 [==============================] - 12s 63ms/step - loss: 0.0763 - accuracy: 0.9709 - val_loss: 0.4700 - val_accuracy: 0.8670
Epoch 15/100
191/191 [==============================] - 12s 63ms/step - loss: 0.0694 - accuracy: 0.9747 - val_loss: 0.6452 - val_accuracy: 0.8557
Epoch 16/100
191/191 [==============================] - 12s 63ms/step - loss: 0.0689 - accuracy: 0.9754 - val_loss: 0.5605 - val_accuracy: 0.8689
Epoch 17/100
191/191 [==============================] - 12s 63ms/step - loss: 0.0684 - accuracy: 0.9710 - val_loss: 0.6006 -

Epoch 15/100
191/191 [==============================] - 12s 63ms/step - loss: 0.0717 - accuracy: 0.9748 - val_loss: 0.4871 - val_accuracy: 0.8803
Epoch 16/100
191/191 [==============================] - 12s 63ms/step - loss: 0.0677 - accuracy: 0.9733 - val_loss: 0.5002 - val_accuracy: 0.8709
Restoring model weights from the end of the best epoch.
Epoch 00016: early stopping
Test Accuracy: 89.44392204284668
Epoch 1/100
191/191 [==============================] - 16s 65ms/step - loss: 0.5080 - accuracy: 0.7562 - val_loss: 0.3622 - val_accuracy: 0.8520
Epoch 2/100
191/191 [==============================] - 12s 61ms/step - loss: 0.2748 - accuracy: 0.8971 - val_loss: 0.3536 - val_accuracy: 0.8662
Epoch 3/100
191/191 [==============================] - 12s 62ms/step - loss: 0.2315 - accuracy: 0.9131 - val_loss: 0.3430 - val_accuracy: 0.8699
Epoch 4/100
191/191 [==============================] - 12s 63ms/step - loss: 0.2040 - accuracy: 0.9250 - val_loss: 0.3905 - val_accuracy: 0.8615
Epoch 5/100

191/191 [==============================] - 13s 67ms/step - loss: 0.1225 - accuracy: 0.9564 - val_loss: 0.4455 - val_accuracy: 0.8660
Epoch 10/100
191/191 [==============================] - 12s 64ms/step - loss: 0.0997 - accuracy: 0.9648 - val_loss: 0.4756 - val_accuracy: 0.8745
Epoch 11/100
191/191 [==============================] - 12s 63ms/step - loss: 0.0927 - accuracy: 0.9645 - val_loss: 0.5187 - val_accuracy: 0.8575
Epoch 12/100
191/191 [==============================] - 12s 63ms/step - loss: 0.0863 - accuracy: 0.9682 - val_loss: 0.5084 - val_accuracy: 0.8604
Epoch 13/100
191/191 [==============================] - 12s 63ms/step - loss: 0.0803 - accuracy: 0.9709 - val_loss: 0.5298 - val_accuracy: 0.8594
Epoch 14/100
191/191 [==============================] - 12s 65ms/step - loss: 0.0725 - accuracy: 0.9746 - val_loss: 0.5178 - val_accuracy: 0.8575
Epoch 15/100
191/191 [==============================] - 14s 73ms/step - loss: 0.0658 - accuracy: 0.9753 - val_loss: 0.6603 - val_accuracy

Epoch 18/100
191/191 [==============================] - 14s 73ms/step - loss: 0.0636 - accuracy: 0.9766 - val_loss: 0.5432 - val_accuracy: 0.8708.0639 - accuracy: 
Epoch 19/100
191/191 [==============================] - 14s 72ms/step - loss: 0.0595 - accuracy: 0.9785 - val_loss: 0.5024 - val_accuracy: 0.8708
Epoch 20/100
191/191 [==============================] - 13s 70ms/step - loss: 0.0528 - accuracy: 0.9791 - val_loss: 0.6161 - val_accuracy: 0.8651
Epoch 21/100
191/191 [==============================] - 13s 66ms/step - loss: 0.0618 - accuracy: 0.9769 - val_loss: 0.5736 - val_accuracy: 0.8849
Restoring model weights from the end of the best epoch.
Epoch 00021: early stopping
Test Accuracy: 88.49056363105774

  Activation Filters       acc1       acc2       acc3       acc4       acc5  \
0       relu       1  89.161170  88.689917  89.066917  89.161170  87.181902   
1       relu       2  88.218659  88.972670  88.312912  88.878417  86.145145   
2       relu       3  88.218659  88.407165 

## Summary

In [31]:
record2.sort_values(by='AVG', ascending=False)

Activation Filters       acc1       acc2       acc3       acc4       acc5  \
4       relu       5  85.956645  87.464654  89.255422  89.161170  86.993402   
2       relu       3  88.218659  88.407165  86.899149  88.312912  90.197927   
1       relu       2  88.218659  88.972670  88.312912  88.878417  86.145145   
5       relu       6  89.443922  86.993402  87.464654  89.161170  87.935907   
3       relu       4  87.747407  88.124412  89.161170  86.427897  88.124412   
0       relu       1  89.161170  88.689917  89.066917  89.161170  87.181902   

        acc6       acc7       acc8       acc9      acc10        AVG  
4  89.443922  88.679248  87.547171  89.433962  89.528304  88.346390  
2  88.784164  88.490564  87.358493  89.716983  87.075472  88.346149  
1  86.993402  89.716983  89.056605  88.018870  88.867927  88.318159  
5  88.595665  89.905661  87.452829  87.735850  88.490564  88.317962  
3  88.689917  89.528304  88.584906  87.264150  88.301885  88.195446  
0  89.066917  88.679248  88.207549  86.886793  85.849059  88.195064

In [32]:
record2[['Activation', 'AVG']].groupby(by='Activation').max().sort_values(by='AVG', ascending=False)

AVG
Activation          
relu        88.34639

In [41]:
report = record2.sort_values(by='AVG', ascending=False)
report = report.to_excel('TCN_MPQA_2.xlsx', sheet_name='static')

# Model 3: Word2Vec - Dynamic

* In this part,  we will fine tune the embeddings while training (dynamic).

## CNN Model

In [34]:
def define_model_3(kernel_size = 3, activation='relu', input_dim = None, 
                   output_dim=300, max_length = None, emb_matrix = None):
    
    inp = Input( shape=(max_length,))
    x = Embedding(input_dim=input_dim, 
                  output_dim=output_dim, 
                  input_length=max_length,
                  # Assign the embedding weight with word2vec embedding marix
                  weights = [emb_matrix],
                  # Set the weight to be not trainable (static)
                  trainable = True)(inp)
    
    x = SpatialDropout1D(0.1)(x)
    
    x = TCN(128,dilations = [1, 2, 4], return_sequences=True, activation = activation, name = 'tcn1')(x)
    x = TCN(64,dilations = [1, 2, 4], return_sequences=True, activation = activation, name = 'tcn2')(x)
    
    avg_pool = GlobalAveragePooling1D()(x)
    max_pool = GlobalMaxPooling1D()(x)
    
    conc = concatenate([avg_pool, max_pool])
    conc = Dense(16, activation="relu")(conc)
    conc = Dropout(0.1)(conc)
    outp = Dense(1, activation="sigmoid")(conc)    

    model = Model(inputs=inp, outputs=outp)
    model.compile( loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
    
    return model

In [35]:
model_0 = define_model_3( input_dim=1000, max_length=100, emb_matrix=np.random.rand(1000, 300))
model_0.summary()

Model: "model_182"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_183 (InputLayer)          [(None, 100)]        0                                            
__________________________________________________________________________________________________
embedding_182 (Embedding)       (None, 100, 300)     300000      input_183[0][0]                  
__________________________________________________________________________________________________
spatial_dropout1d_182 (SpatialD (None, 100, 300)     0           embedding_182[0][0]              
__________________________________________________________________________________________________
tcn1 (TCN)                      (None, 100, 128)     279936      spatial_dropout1d_182[0][0]      
__________________________________________________________________________________________

## Train and Test the Model

In [36]:
class myCallback(tf.keras.callbacks.Callback):
    # Overide the method on_epoch_end() for our benefit
    def on_epoch_end(self, epoch, logs={}):
        if (logs.get('accuracy') > 0.93):
            print("\nReached 93% accuracy so cancelling training!")
            self.model.stop_training=True

callbacks = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', min_delta=0, 
                                             patience=10, verbose=2, 
                                             mode='auto', restore_best_weights=True)

In [37]:
# Parameter Initialization
trunc_type='post'
padding_type='post'
oov_tok = "<UNK>"
activations = ['relu']
print('Loading embedding statistics . . .')
emb_mean = emb_mean
emb_std = emb_std
print('Done!')
kernel_sizes = [1, 2, 3, 4, 5, 6]

columns = ['Activation', 'Filters', 'acc1', 'acc2', 'acc3', 'acc4', 'acc5', 'acc6', 'acc7', 'acc8', 'acc9', 'acc10', 'AVG']
record3 = pd.DataFrame(columns = columns)

# prepare cross validation with 10 splits and shuffle = True
kfold = KFold(10, True)

# Separate the sentences and the labels
sentences, labels = list(corpus.sentence), list(corpus.label)

exp = 0

for activation in activations:
    for kernel_size in kernel_sizes:
        exp+=1
        print('-------------------------------------------')
        print('Training {}: {} activation, {} kernel size.'.format(exp, activation, kernel_size))
        print('-------------------------------------------')
        
        # kfold.split() will return set indices for each split
        acc_list = []
        for train, test in kfold.split(sentences):
            
            train_x, test_x = [], []
            train_y, test_y = [], []
            
            for i in train:
                train_x.append(sentences[i])
                train_y.append(labels[i])

            for i in test:
                test_x.append(sentences[i])
                test_y.append(labels[i])

            # Turn the labels into a numpy array
            train_y = np.array(train_y)
            test_y = np.array(test_y)

            # encode data using
            # Cleaning and Tokenization
            tokenizer = Tokenizer(oov_token=oov_tok)
            tokenizer.fit_on_texts(train_x)

            # Turn the text into sequence
            training_sequences = tokenizer.texts_to_sequences(train_x)
            test_sequences = tokenizer.texts_to_sequences(test_x)

            max_len = max_length(training_sequences)

            # Pad the sequence to have the same size
            Xtrain = pad_sequences(training_sequences, maxlen=max_len, padding=padding_type, truncating=trunc_type)
            Xtest = pad_sequences(test_sequences, maxlen=max_len, padding=padding_type, truncating=trunc_type)

            word_index = tokenizer.word_index
            vocab_size = len(word_index)+1
            
            
            emb_matrix = pretrained_embedding_matrix(word2vec, word_index, emb_mean, emb_std)
            
            # Define the input shape
            model = define_model_3(kernel_size, activation, input_dim=vocab_size, 
                                 max_length=max_len, emb_matrix=emb_matrix)

            # Train the model
            model.fit(Xtrain, train_y, batch_size=50, epochs=100, verbose=1, 
                      callbacks=[callbacks], validation_data=(Xtest, test_y))

            # evaluate the model
            loss, acc = model.evaluate(Xtest, test_y, verbose=0)
            print('Test Accuracy: {}'.format(acc*100))

            acc_list.append(acc*100)
            
        mean_acc = np.array(acc_list).mean()
        parameters = [activation, kernel_size]
        entries = parameters + acc_list + [mean_acc]

        temp = pd.DataFrame([entries], columns=columns)
        record3 = record3.append(temp, ignore_index=True)
        print()
        print(record3)
        print()

Loading embedding statistics . . .
Done!
-------------------------------------------
Training 1: relu activation, 1 kernel size.
-------------------------------------------


C:\Users\Diardano Raihan\Anaconda3\envs\tf-gpu\lib\site-packages\sklearn\utils\validation.py:72: FutureWarning: Pass shuffle=True as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  "will result in an error", FutureWarning)


Epoch 1/100
191/191 [==============================] - 25s 103ms/step - loss: 0.4910 - accuracy: 0.7565 - val_loss: 0.3082 - val_accuracy: 0.8728
Epoch 2/100
191/191 [==============================] - 18s 92ms/step - loss: 0.2205 - accuracy: 0.9192 - val_loss: 0.3091 - val_accuracy: 0.8765
Epoch 3/100
191/191 [==============================] - 18s 97ms/step - loss: 0.1584 - accuracy: 0.9453 - val_loss: 0.3444 - val_accuracy: 0.8709
Epoch 4/100
191/191 [==============================] - 18s 94ms/step - loss: 0.1215 - accuracy: 0.9562 - val_loss: 0.3840 - val_accuracy: 0.8624
Epoch 5/100
191/191 [==============================] - 17s 91ms/step - loss: 0.1075 - accuracy: 0.9615 - val_loss: 0.4293 - val_accuracy: 0.8596
Epoch 6/100
191/191 [==============================] - 19s 97ms/step - loss: 0.0775 - accuracy: 0.9721 - val_loss: 0.6781 - val_accuracy: 0.8483
Epoch 7/100
191/191 [==============================] - 18s 92ms/step - loss: 0.0732 - accuracy: 0.9739 - val_loss: 0.5809 - val_a

191/191 [==============================] - 19s 95ms/step - loss: 0.0414 - accuracy: 0.9825 - val_loss: 0.6780 - val_accuracy: 0.8858
Epoch 14/100
191/191 [==============================] - 17s 91ms/step - loss: 0.0401 - accuracy: 0.9833 - val_loss: 0.4802 - val_accuracy: 0.8792
Epoch 15/100
191/191 [==============================] - 17s 91ms/step - loss: 0.0389 - accuracy: 0.9840 - val_loss: 0.8897 - val_accuracy: 0.8547
Epoch 16/100
191/191 [==============================] - 17s 91ms/step - loss: 0.0371 - accuracy: 0.9844 - val_loss: 0.7960 - val_accuracy: 0.8575
Epoch 17/100
191/191 [==============================] - 17s 91ms/step - loss: 0.0370 - accuracy: 0.9849 - val_loss: 0.8226 - val_accuracy: 0.8642
Epoch 18/100
191/191 [==============================] - 17s 91ms/step - loss: 0.0274 - accuracy: 0.9879 - val_loss: 0.9531 - val_accuracy: 0.8491
Epoch 19/100
191/191 [==============================] - 17s 91ms/step - loss: 0.0301 - accuracy: 0.9868 - val_loss: 0.8014 - val_accuracy

191/191 [==============================] - 17s 89ms/step - loss: 0.0444 - accuracy: 0.9822 - val_loss: 0.9322 - val_accuracy: 0.8454
Epoch 13/100
191/191 [==============================] - 17s 89ms/step - loss: 0.0318 - accuracy: 0.9864 - val_loss: 1.1449 - val_accuracy: 0.8539
Epoch 14/100
191/191 [==============================] - 17s 89ms/step - loss: 0.0444 - accuracy: 0.9829 - val_loss: 1.1557 - val_accuracy: 0.8577
Epoch 15/100
191/191 [==============================] - 17s 89ms/step - loss: 0.0318 - accuracy: 0.9858 - val_loss: 0.7810 - val_accuracy: 0.8445
Epoch 16/100
191/191 [==============================] - 17s 89ms/step - loss: 0.0406 - accuracy: 0.9843 - val_loss: 1.1746 - val_accuracy: 0.8671
Epoch 17/100
191/191 [==============================] - 17s 89ms/step - loss: 0.0307 - accuracy: 0.9859 - val_loss: 0.8103 - val_accuracy: 0.8549
Restoring model weights from the end of the best epoch.
Epoch 00017: early stopping
Test Accuracy: 87.27615475654602
Epoch 1/100
191/191 

...

191/191 [==============================] - 17s 89ms/step - loss: 0.0598 - accuracy: 0.9763 - val_loss: 0.5447 - val_accuracy: 0.8558
Epoch 10/100
191/191 [==============================] - 17s 89ms/step - loss: 0.0500 - accuracy: 0.9820 - val_loss: 0.5135 - val_accuracy: 0.8709
Epoch 11/100
191/191 [==============================] - 17s 87ms/step - loss: 0.0473 - accuracy: 0.9826 - val_loss: 0.5586 - val_accuracy: 0.8671
Epoch 12/100
191/191 [==============================] - 17s 89ms/step - loss: 0.0429 - accuracy: 0.9816 - val_loss: 0.5622 - val_accuracy: 0.8728
Restoring model weights from the end of the best epoch.
Epoch 00012: early stopping
Test Accuracy: 89.34966921806335
Epoch 1/100
191/191 [==============================] - 22s 92ms/step - loss: 0.7308 - accuracy: 0.6959 - val_loss: 0.3396 - val_accuracy: 0.8690
Epoch 2/100
191/191 [==============================] - 17s 87ms/step - loss: 0.2408 - accuracy: 0.9141 - val_loss: 0.3127 - val_accuracy: 0.8860
Epoch 3/100
191/191 [=

191/191 [==============================] - 17s 89ms/step - loss: 0.0474 - accuracy: 0.9804 - val_loss: 0.6341 - val_accuracy: 0.8528
Epoch 10/100
191/191 [==============================] - 17s 90ms/step - loss: 0.0498 - accuracy: 0.9809 - val_loss: 0.7742 - val_accuracy: 0.8509
Epoch 11/100
191/191 [==============================] - 17s 89ms/step - loss: 0.0435 - accuracy: 0.9830 - val_loss: 0.9174 - val_accuracy: 0.8434
Epoch 12/100
191/191 [==============================] - 17s 89ms/step - loss: 0.0359 - accuracy: 0.9841 - val_loss: 1.0030 - val_accuracy: 0.8509
Restoring model weights from the end of the best epoch.
Epoch 00012: early stopping
Test Accuracy: 86.88679337501526

  Activation Filters       acc1       acc2       acc3       acc4       acc5  \
0       relu       1  87.653160  89.255422  88.124412  88.312912  88.312912   
1       relu       2  89.161170  87.276155  87.181902  89.066917  88.312912   
2       relu       3  88.501412  89.632422  85.768145  87.653160  87.74740

Epoch 7/100
191/191 [==============================] - 17s 89ms/step - loss: 0.0793 - accuracy: 0.9728 - val_loss: 0.7792 - val_accuracy: 0.8812
Epoch 8/100
191/191 [==============================] - 17s 90ms/step - loss: 0.0719 - accuracy: 0.9731 - val_loss: 0.5159 - val_accuracy: 0.8718
Epoch 9/100
191/191 [==============================] - 17s 89ms/step - loss: 0.0610 - accuracy: 0.9783 - val_loss: 0.7746 - val_accuracy: 0.8850
Epoch 10/100
191/191 [==============================] - 17s 88ms/step - loss: 0.0537 - accuracy: 0.9801 - val_loss: 0.7883 - val_accuracy: 0.8878
Epoch 11/100
191/191 [==============================] - 17s 88ms/step - loss: 0.0451 - accuracy: 0.9826 - val_loss: 0.7885 - val_accuracy: 0.8869
Epoch 12/100
191/191 [==============================] - 17s 87ms/step - loss: 0.0440 - accuracy: 0.9827 - val_loss: 1.0199 - val_accuracy: 0.8831
Epoch 13/100
191/191 [==============================] - 17s 87ms/step - loss: 0.0414 - accuracy: 0.9834 - val_loss: 1.0338 - va

191/191 [==============================] - 17s 87ms/step - loss: 0.1400 - accuracy: 0.9493 - val_loss: 0.3661 - val_accuracy: 0.8699
Epoch 4/100
191/191 [==============================] - 17s 87ms/step - loss: 0.1038 - accuracy: 0.9645 - val_loss: 0.3844 - val_accuracy: 0.8831
Epoch 5/100
191/191 [==============================] - 17s 87ms/step - loss: 0.0885 - accuracy: 0.9680 - val_loss: 0.4253 - val_accuracy: 0.8775
Epoch 6/100
191/191 [==============================] - 17s 87ms/step - loss: 0.0697 - accuracy: 0.9741 - val_loss: 0.4907 - val_accuracy: 0.8417
Epoch 7/100
191/191 [==============================] - 17s 87ms/step - loss: 0.0677 - accuracy: 0.9737 - val_loss: 0.5721 - val_accuracy: 0.8633
Epoch 8/100
191/191 [==============================] - 17s 88ms/step - loss: 0.0556 - accuracy: 0.9778 - val_loss: 0.6303 - val_accuracy: 0.8624
Epoch 9/100
191/191 [==============================] - 17s 87ms/step - loss: 0.0527 - accuracy: 0.9785 - val_loss: 0.5904 - val_accuracy: 0.86

191/191 [==============================] - 17s 88ms/step - loss: 0.0632 - accuracy: 0.9762 - val_loss: 0.6062 - val_accuracy: 0.8745
Epoch 9/100
191/191 [==============================] - 17s 88ms/step - loss: 0.0531 - accuracy: 0.9796 - val_loss: 0.6110 - val_accuracy: 0.8679
Epoch 10/100
191/191 [==============================] - 17s 89ms/step - loss: 0.0491 - accuracy: 0.9827 - val_loss: 0.6932 - val_accuracy: 0.8604
Epoch 11/100
191/191 [==============================] - 17s 89ms/step - loss: 0.0524 - accuracy: 0.9823 - val_loss: 0.6726 - val_accuracy: 0.8745
Epoch 12/100
191/191 [==============================] - 17s 88ms/step - loss: 0.0377 - accuracy: 0.9837 - val_loss: 0.8599 - val_accuracy: 0.8623
Epoch 13/100
191/191 [==============================] - 17s 88ms/step - loss: 0.0508 - accuracy: 0.9835 - val_loss: 0.7200 - val_accuracy: 0.8491
Epoch 14/100
191/191 [==============================] - 17s 89ms/step - loss: 0.0361 - accuracy: 0.9853 - val_loss: 0.9041 - val_accuracy:

## Summary

In [38]:
record3.sort_values(by='AVG', ascending=False)

Activation Filters       acc1       acc2       acc3       acc4       acc5  \
2       relu       3  88.501412  89.632422  85.768145  87.653160  87.747407   
3       relu       4  90.103674  89.349669  88.595665  87.370408  88.030159   
0       relu       1  87.653160  89.255422  88.124412  88.312912  88.312912   
5       relu       6  87.181902  88.312912  87.370408  88.312912  86.899149   
4       relu       5  88.407165  87.087655  88.218659  88.784164  87.464654   
1       relu       2  89.161170  87.276155  87.181902  89.066917  88.312912   

        acc6       acc7       acc8       acc9      acc10        AVG  
2  89.820921  87.830186  88.962263  86.886793  88.207549  88.101026  
3  87.747407  86.603773  87.075472  87.641507  86.886793  87.940453  
0  87.747407  86.509436  87.075472  88.584906  87.547171  87.912321  
5  86.710650  89.150941  86.981130  88.584906  87.547171  87.705208  
4  87.935907  86.981130  86.792451  87.830186  87.169814  87.667179  
1  87.181902  68.867922  88.207549  87.924528  87.641507  86.082246

In [40]:
report = record3.sort_values(by='AVG', ascending=False)
report = report.to_excel('TCN_MPQA_3.xlsx', sheet_name='dynamic')